# 2) Compute features for random parameters

Using the generated random parameters, we next compute responses and features for 3 different feature sets:

- 'bap': features extracted from somatic recording with 3 different steps + a pulse response measured at the soma plus two locations on the apical dendrite (Backpropagating Action Potential)

- 'soma': features extracted from somatic recording with 3 different steps

- 'extra': features extracted from somatic recording with 3 different steps and from mean extracellular action potential

Computed features are saved in the `config/features/` folder for each parameter set and are ready to be used for optimization.

In [ ]:
%load_ext autoreload
%autoreload

import time
import json

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path

import MEAutility as mu
import bluepyopt.ephys as ephys

%matplotlib notebook

In [ ]:
import model
import evaluator

In [ ]:
np.random.seed(2308)

### Define extracellular electrodes

Importantly, if the electrode design changes, new features need to be generated. 

However, a subset of channels can be selected before running the optimization procedure.

In [ ]:
mea_type = 'planar'  # 'linear' or 'planar'

probe, electrode = model.define_electrode(
    probe_type=mea_type,
    num_linear=20, 
    linear_span=[-500, 1000], 
    z_shift=20, 
    probe_center=[0, 200, 20],
    mea_dim=[20, 4],
    mea_pitch=[70, 35],
)

In [ ]:
ax = mu.plot_probe(probe)

### Compute features

Features for the different feature sets ('bap', 'soma', 'extra') are listed in the `config/feature_list.json` file.

Here we loop through the different feature sets and random parameters, compute the corresponding features and save them in a `json` file that will be later used to construct the `CellEvaluator`.


In [ ]:
random_params_file = 'config/params/smart_random.csv'
random_params = pd.read_csv(random_params_file, index_col='index')
random_params

In [ ]:
save_features_and_probe = True

In [ ]:
feature_sets = ["extra", "soma", "multiple"]
channels = 'map'

morphology = ephys.morphologies.NrnFileMorphology('morphology/cell1.asc', do_replace_axon=True)
param_configs = json.load(open('config/parameters.json'))
parameters = model.define_parameters()
mechanisms = model.define_mechanisms()

l5pc_cell = ephys.models.LFPyCellModel('l5pc', 
                                       v_init=-65., 
                                       morph=morphology, 
                                       mechs=mechanisms, 
                                       params=parameters)

param_names = [param.name for param in l5pc_cell.params.values() if not param.frozen]      
responses = {}
features = {}

protocols = evaluator.define_protocols()
protocols_extra = evaluator.define_protocols(electrode=electrode) 

simulator = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True)
simulator_extra = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True, electrode=electrode)

for feature_set in feature_sets:
    
    print(f'Feature set {feature_set}')

    if feature_set == "extra":
        fitness_protocols = protocols_extra
        sim = simulator_extra
    else:
        fitness_protocols = protocols
        sim = simulator

    responses[feature_set] = []
    features[feature_set] = []
    
    for i, (index, params) in enumerate(random_params.iterrows()):
        
        print(f'{i+1} / {len(random_params)}, {index}')   
        
        t_start = time.time()
        response, feat_dict = evaluator.compute_feature_values(
            params, 
            l5pc_cell, 
            fitness_protocols, 
            sim,
            feature_set=feature_set, 
            probe=probe, 
            channels=channels,
            verbose=True
        )
        print(f"Elapsed time: {time.time() - t_start}")
        
        features[feature_set].append(feat_dict)
        responses[feature_set].append(response)
        
        if save_features_and_probe:
            
            feature_folder = Path("config/features/")
            if channels is 'map':
                feature_file = feature_folder / f"{index}_{mea_type}_map" / f'{feature_set}.pkl'
            else:
                feature_file = feature_folder / str(index) / f'{feature_set}.pkl'

            feature_file.parent.mkdir(parents=True, exist_ok=True)

            with feature_file.open('wb') as f:
                pickle.dump(feat_dict, f)

            with (feature_file.parent / 'probe.json').open('w') as f:
                json.dump(probe.info, f, indent=4)

            print(f"Saved to {feature_file}")

In [ ]:
pos = features["extra"][0]['Step1']['MEA']['pos_peak_relative'][0]

In [ ]:
l5pc_plot.plot_feature_map(pos, probe)

In [ ]:
eap = l5pc_evaluator.calculate_eap(responses=responses['extra'][0], protocols=fitness_protocols,
                                   protocol_name='Step1')

In [ ]:
eap

In [ ]:
fig = l5pc_plot.plot_multiple_responses(responses['bap'], return_fig=True)

In [ ]:
eaps = []
for i, resp in enumerate(responses['extra']):
    eap = l5pc_evaluator.calculate_eap(resp, protocol_name='Step1', protocols=fitness_protocols)
    eaps.append(eap)

In [ ]:
eaps = np.array(eaps)
eaps.shape

In [ ]:
mu.plot_mea_recording(eaps, probe)